In [4]:
import os
from pathlib import Path, PurePath
import requests
import numpy as np
import pandas as pd
pd.set_option('max_columns', 50)
pd.set_option('max_rows', 9999)
from bs4 import BeautifulSoup
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Download data from NYC's Dept of Finance website

In [5]:
def get_sales_links(url, skip_table):
    """
    Get links to excel spreadsheets by year (from 2003 to 2021) and 
    borough from NYC Department of Finance website.
    Two seperate lists are created due to the change in which row
    column headers begin after 2010.
    """
    try:
        website_text = requests.get(url).text
        soup = BeautifulSoup(website_text)
        tables = soup.find_all('table')
        
        links = [each_link['href']
                for each_table in tables[skip_table:]
                for each_link in each_table.select("a[href*='.xls']")]

        list_of_urls = ["https://www1.nyc.gov" + each for each in links]

    except requests.RequestException as exception:
        return exception

    return list_of_urls

def check_for_directory(dirname):
    """
    Check the parent directory for a directory.
    One is created if it doesn't exist to stage files
    """
    parent_dir = Path(os.getenv("PARENT_DIR"))
    join_dirs = parent_dir.joinpath(dirname)
    if not Path.is_dir(join_dirs):
        Path.mkdir(join_dirs)


In [6]:
#Downloading rolling sales data for 2003 through 2021

parent_dir = Path(os.getenv("PARENT_DIR"))
data_path = parent_dir.joinpath('data')
raw_path = data_path.joinpath("raw")
stage_path = data_path.joinpath("stage")
production_path = data_path.joinpath("production")

check_for_directory(data_path)
check_for_directory(raw_path)
check_for_directory(production_path)

SALES_URL = "https://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page"        
url_links = get_sales_links(SALES_URL, 1)

for each_link in url_links:
    response = requests.get(each_link)
    filename = PurePath(each_link).name
    filepath = raw_path.joinpath(filename)
    with open(filepath, mode='wb') as f:
        f.write(response.content)

In [7]:
#Downloading rolling sales data for 2022

SALES_URL_2022 = "https://www.nyc.gov/site/finance/taxes/property-rolling-sales-data.page"


url_2022_links = get_sales_links(SALES_URL_2022, 0)

for each_2022_link in url_2022_links:
    response = requests.get(each_2022_link)
    filename = PurePath(each_2022_link).name
    filepath = raw_path.joinpath(filename)
    with open(filepath, mode='wb') as f:
        f.write(response.content)

In [8]:
# List out all files to batch them into groups
filenames = os.listdir(raw_path)
filenames.sort()
filenames = [str(raw_path) + "/" + str(filename) for filename in filenames]
print('\n'.join(filenames))


/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_bronx.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_brooklyn.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_manhattan.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_queens.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_statenisland.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_bronx.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_brooklyn.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_manhattan.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_queens.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_statenisland.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_bronx.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_brooklyn.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_manhattan.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_queens.xls
/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_statenisland

In [9]:
# Batching spreadsheets by number of top line rows that should be skipped

all2022_sales = ["/Users/joshuaacosta/documents/nyc_sales/data/raw/rollingsales_bronx.xlsx",
                 "/Users/joshuaacosta/documents/nyc_sales/data/raw/rollingsales_brooklyn.xlsx",
                 "/Users/joshuaacosta/documents/nyc_sales/data/raw/rollingsales_manhattan.xlsx",
                 "/Users/joshuaacosta/documents/nyc_sales/data/raw/rollingsales_queens.xlsx",
                 "/Users/joshuaacosta/documents/nyc_sales/data/raw/rollingsales_statenisland.xlsx"]

all2003_2010_sales = ["/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2007_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2007_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2007_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2007_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2007_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2008_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2008_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2008_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2008_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_2008_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_bronx_03.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_bronx_04.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_bronx_05.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_bronx_06.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_brooklyn_03.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_brooklyn_04.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_brooklyn_05.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_brooklyn_06.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_manhattan_03.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_manhattan_04.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_manhattan_05.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_manhattan_06.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_queens_03.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_queens_04.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_queens_05.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_queens_06.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_si_03.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_si_04.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_si_05.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/sales_si_06.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2009_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2010_statenisland.xls"]

all2011_2019_sales = ["/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2011_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2012_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2012_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2012_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2012_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2012_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2013_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2013_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2013_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2013_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2013_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2014_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2014_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2014_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2014_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2014_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2015_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2015_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2015_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2015_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2015_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2016_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2016_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2016_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2016_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2016_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2017_bronx.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2017_brooklyn.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2017_manhattan.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2017_queens.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2017_statenisland.xls",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2018_bronx.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2018_brooklyn.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2018_manhattan.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2018_queens.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2018_statenisland.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2019_bronx.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2019_brooklyn.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2019_manhattan.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2019_queens.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2019_statenisland.xlsx"]

all2020_2021_sales = ["/Users/joshuaacosta/documents/nyc_sales/data/raw/2020_bronx.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2020_brooklyn.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2020_manhattan.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2020_queens.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2020_staten_island.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2021_bronx.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2021_brooklyn.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2021_manhattan.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2021_queens.xlsx",
                      "/Users/joshuaacosta/documents/nyc_sales/data/raw/2021_staten_island.xlsx"]

## Reading files into pandas

In [10]:
def read_excel_data(list_of_filenames, skip_rows_num):
    """
    Creates a list of dataframes, one df per spreadsheet, and then concats them into
    one combined dataframe. For each spreadsheet, columns desired are specified.
    Tax related columns and those with mostly null values are not added.
    Numeber of rows to skip for each df is specific due to change made by NY DOF 
    after 2010.
    """
    
    use_col_names = ["BOROUGH", "NEIGHBORHOOD","BUILDING CLASS CATEGORY", 
                     "TAX CLASS AS OF FINAL ROLL", "BLOCK", "LOT", "EASEMENT",
                     "BUILDING CLASS AS OF FINAL ROLL", "ADDRESS", "APARTMENT NUMBER", 
                     "ZIP CODE","RESIDENTIAL UNITS", "COMMERCIAL UNITS", "TOTAL UNITS",
                     "LAND SQUARE FEET", "GROSS SQUARE FEET", "YEAR BUILT",
                     "TAX CLASS AT TIME OF SALE",
                     "BUILDING CLASS AT TIME OF SALE",
                     "SALE PRICE", "SALE DATE"]

    list_of_dfs = [pd.read_excel(filename, skiprows=skip_rows_num,
                   dtype=str, names=use_col_names) for filename in list_of_filenames]

    dataframe = pd.concat(list_of_dfs, ignore_index=True, sort=False)

    return dataframe

In [11]:
#Reading sales data in Pandas

sales_2011_2019_df = read_excel_data(all2011_2019_sales, 4)
sales_2003_2010_df = read_excel_data(all2003_2010_sales, 3)
sales_2020_2021_df = read_excel_data(all2020_2021_sales, 6)
sales_2022_df = read_excel_data(all2022_sales, 4)

combined_df = pd.concat([sales_2011_2019_df, sales_2003_2010_df,
                         sales_2020_2021_df, sales_2022_df],
                        ignore_index=True, sort=False)

In [12]:
combined_df.shape

(1855333, 21)

In [13]:
#Save dataframe combining all spreadsheets into a csv in stage directory
combined_df.to_csv((str(stage_path) + "/" + "combined_all_sales.csv"), index=False, header=True)

## Cleaning Sales Data

In [14]:
#Read unclean csv file
all_sales_csv = str(stage_path) + "/" + "combined_all_sales.csv"

sales_df = pd.read_csv(all_sales_csv, skipinitialspace=True, low_memory=False)

sales_df.shape

(1855333, 21)

In [15]:
sales_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1855333 entries, 0 to 1855332
Data columns (total 21 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   BOROUGH                          float64
 1   NEIGHBORHOOD                     object 
 2   BUILDING CLASS CATEGORY          object 
 3   TAX CLASS AS OF FINAL ROLL       object 
 4   BLOCK                            float64
 5   LOT                              float64
 6   EASEMENT                         object 
 7   BUILDING CLASS AS OF FINAL ROLL  object 
 8   ADDRESS                          object 
 9   APARTMENT NUMBER                 object 
 10  ZIP CODE                         float64
 11  RESIDENTIAL UNITS                float64
 12  COMMERCIAL UNITS                 float64
 13  TOTAL UNITS                      float64
 14  LAND SQUARE FEET                 float64
 15  GROSS SQUARE FEET                float64
 16  YEAR BUILT                       float64
 17  TAX CLAS

In [16]:
#Drop duplicate rows if any

sales_df.drop_duplicates()
sales_df.shape

(1855333, 21)

In [17]:
"""
Updates columns names by stripping white spaces, makes all characters
lower case and replacing and spaces with an underscore.
"""

sales_df.columns = sales_df.columns.str.strip().str.lower().str.replace(' ', '_')
sales_df.columns

Index(['borough', 'neighborhood', 'building_class_category',
       'tax_class_as_of_final_roll', 'block', 'lot', 'easement',
       'building_class_as_of_final_roll', 'address', 'apartment_number',
       'zip_code', 'residential_units', 'commercial_units', 'total_units',
       'land_square_feet', 'gross_square_feet', 'year_built',
       'tax_class_at_time_of_sale', 'building_class_at_time_of_sale',
       'sale_price', 'sale_date'],
      dtype='object')

In [18]:
#Drop columns with null values and/or duplicate
sales_df.drop(['easement'], axis=1, inplace=True)
sales_df.shape

(1855333, 20)

In [19]:
#Drop all property sales with a sale price under $125,000
sales_df = sales_df[sales_df ["sale_price"] >= 125000.00]
sales_df.shape

(1176034, 20)

In [20]:
#clean up neighborhoods names by stripping whitespace
before_neighborhood_num = sales_df.neighborhood.nunique()

sales_df.neighborhood = sales_df.neighborhood.apply(str.rstrip)

after_neighborhood_num = sales_df.neighborhood.nunique()

print(f"Number of unique neighborhood names before stripping: {before_neighborhood_num} ")
print(f"Number of unique neighborhood names after stripping: {after_neighborhood_num} ")

Number of unique neighborhood names before stripping: 513 
Number of unique neighborhood names after stripping: 263 


In [21]:
#Updating a neighborhood name that had incorrect value
sales_df["neighborhood"].replace('3004', 'BATH BEACH', inplace=True)

In [22]:
#update year built of property address for accuracy
sales_df.loc[sales_df.address == "762 MARCY AVENUE, 1B", 'year_built'] = 2017.0
sales_df.loc[sales_df.address == "762 MARCY AVENUE, 4", 'year_built'] = 2017.0
sales_df.loc[sales_df.address == "9 BARTLETT AVENUE, 0", 'year_built'] = 2019.0

In [23]:
#Update zip code on property for accuracy 
sales_df.loc[sales_df.address == "762 MARCY AVENUE, 1B", 'zip_code'] = 11216.0
sales_df.loc[sales_df.address == "762 MARCY AVENUE, 4", 'zip_code'] = 11216.0

In [24]:
"""
Split apartment number contained inside the address column
and update the apartment_number column.

Slice original dataframe to a another dataframe if the address
value for a row contains a comma. Splits address value into two
values if initial cell contained the column: one containing the
property address and another containing apartment number.
Apartment value then added to apartment_number column for row.
"""

df2 = sales_df[sales_df['address'].str.contains(",",na=False)]
for index_label, row_series in df2.iterrows():
    new_address, apt_num = row_series['address'].split(',', 1)
    df2.at[index_label, 'address'] = new_address
    df2.at[index_label, 'apartment_number'] = apt_num
sales_df.update(df2)

In [25]:
#removes all commas from cells in the apartment_number column
sales_df["apartment_number"] = sales_df["apartment_number"].str.replace(',', '')

In [26]:
#fill nans in sepecific columns

sales_df.replace([np.inf, -np.inf], np.nan)
fill_zero = {"residential_units":0.0, "commercial_units":0.0, "total_units":0.0,\
             "land_square_feet":0.0, "gross_square_feet":0.0, "zip_code": 0.0, \
             "year_built": 0.0, "building_class_category":"0"}
sales_df.fillna(value=fill_zero, inplace=True)

In [27]:
#update dtypes for several columns to minimize memory size

sales_df["borough"] = sales_df["borough"].astype("int8")
sales_df["zip_code"] = sales_df["zip_code"].astype("int16")
sales_df["block"] = sales_df["block"].astype("int16")
sales_df["lot"] = sales_df["lot"].astype("int16")
sales_df["residential_units"] = sales_df["residential_units"].astype("int16")
sales_df["commercial_units"] = sales_df["commercial_units"].astype("int16")
sales_df["total_units"] = sales_df["total_units"].astype("int16")
sales_df["land_square_feet"] = sales_df["land_square_feet"].astype("int64")
sales_df["gross_square_feet"] = sales_df["gross_square_feet"].astype("int64")
sales_df["year_built"] = sales_df["year_built"].astype("int16")
sales_df["tax_class_at_time_of_sale"] = sales_df["tax_class_at_time_of_sale"].astype("int8")
sales_df["sale_price"] = sales_df["sale_price"].astype("float64")
sales_df["sale_date"] = sales_df["sale_date"].astype("datetime64[ns]")

In [28]:
""" 
Replaces double spaces with one space in strings and 
combines multiple names for the same category.
"""
before_category_num = sales_df.building_class_category.nunique()

sales_df["building_class_category"] = sales_df["building_class_category"].apply(str.rstrip)

sales_df["building_class_category"] = sales_df["building_class_category"].str.replace('  ',' ')

replacement_values = {'01 ONE FAMILY DWELLINGS': '01 ONE FAMILY HOMES',
                  '02 TWO FAMILY DWELLINGS': '02 TWO FAMILY HOMES',
                  '03 THREE FAMILY DWELLINGS': '03 THREE FAMILY HOMES',
                  '17 CONDOPS': '17 CONDO COOPS',
                  '18 TAX CLASS 3 - UNTILITY PROPERTIES': '18 TAX CLASS 3 - UTILITY PROPERTIES'}

sales_df["building_class_category"] = sales_df["building_class_category"].replace(replacement_values)

after_category_num = sales_df.building_class_category.nunique()

print(f"Number of unique building categories before stripping: {before_category_num} ")
print(f"Number of unique building categories after stripping: {after_category_num} ")

Number of unique building categories before stripping: 144 
Number of unique building categories after stripping: 49 


In [29]:
b = sales_df["building_class_category"].unique()
b.sort()
print(b)

['0' '01 ONE FAMILY HOMES' '02 TWO FAMILY HOMES' '03 THREE FAMILY HOMES'
 '04 TAX CLASS 1 CONDOS' '05 TAX CLASS 1 VACANT LAND'
 '06 TAX CLASS 1 - OTHER' '07 RENTALS - WALKUP APARTMENTS'
 '08 RENTALS - ELEVATOR APARTMENTS' '09 COOPS - WALKUP APARTMENTS'
 '10 COOPS - ELEVATOR APARTMENTS' '11 SPECIAL CONDO BILLING LOTS'
 '11A CONDO-RENTALS' '12 CONDOS - WALKUP APARTMENTS'
 '13 CONDOS - ELEVATOR APARTMENTS' '14 RENTALS - 4-10 UNIT'
 '15 CONDOS - 2-10 UNIT RESIDENTIAL'
 '16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT' '17 CONDO COOPS'
 '18 TAX CLASS 3 - UTILITY PROPERTIES' '21 OFFICE BUILDINGS'
 '22 STORE BUILDINGS' '23 LOFT BUILDINGS'
 '24 TAX CLASS 4 - UTILITY BUREAU PROPERTIES' '25 LUXURY HOTELS'
 '26 OTHER HOTELS' '27 FACTORIES' '28 COMMERCIAL CONDOS'
 '29 COMMERCIAL GARAGES' '30 WAREHOUSES' '31 COMMERCIAL VACANT LAND'
 '32 HOSPITAL AND HEALTH FACILITIES' '33 EDUCATIONAL FACILITIES'
 '34 THEATRES' '35 INDOOR PUBLIC AND CULTURAL FACILITIES'
 '36 OUTDOOR RECREATIONAL FACILITIES' '37 RELIGIOUS

In [30]:
sales_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176034 entries, 1 to 1855332
Data columns (total 20 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   borough                          1176034 non-null  int8          
 1   neighborhood                     1176034 non-null  object        
 2   building_class_category          1176034 non-null  object        
 3   tax_class_as_of_final_roll       1166004 non-null  object        
 4   block                            1176034 non-null  int16         
 5   lot                              1176034 non-null  int16         
 6   building_class_as_of_final_roll  1166004 non-null  object        
 7   address                          1176033 non-null  object        
 8   apartment_number                 547930 non-null   object        
 9   zip_code                         1176034 non-null  int16         
 10  residential_units             

In [31]:
sales_df.head(5)

,borough,neighborhood,building_class_category,tax_class_as_of_final_roll,block,lot,building_class_as_of_final_roll,address,apartment_number,zip_code,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date
1,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,64,A1,467 EAST 185 STREET,NaN,10458,1,0,1,1667,1296,1910,1,A1,329000.0,2011-01-19
2,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,A1,2085 BATHGATE AV,NaN,10457,1,0,1,2060,1629,1899,1,A1,288500.0,2011-06-29
5,2,BATHGATE,02 TWO FAMILY HOMES,1,2929,128,B1,559 CLAREMONT PARKWAY,NaN,10457,2,0,2,2361,2394,1995,1,B1,365000.0,2011-07-13
6,2,BATHGATE,02 TWO FAMILY HOMES,1,3030,76,B2,4435 PARK AVENUE,NaN,10457,2,0,2,2444,2125,1899,1,B2,375000.0,2011-05-18
8,2,BATHGATE,02 TWO FAMILY HOMES,1,3036,43,B1,2057 WASHINGTON AVENUE,NaN,10457,2,0,2,3323,3438,1997,1,B1,360000.0,2011-06-29


In [50]:
# Write dataframe to csv by sale data year

list_of_years = range(2003, 2023)

sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
sales_df['year'] = sales_df['sale_date'].dt.strftime('%Y')

for each_year in list_of_years:
    temp_df = sales_df[sales_df.year == str(each_year)]
    temp_df.to_csv((str(production_path) + "/" + f"{each_year}_cleaned_125k+_sales.csv"), index=False, header=True)


(70076, 21)
(81294, 21)
(81557, 21)
(75716, 21)
(70001, 21)
(55151, 21)
(42821, 21)
(39784, 21)
(43819, 21)
(47949, 21)
(55218, 21)
(55502, 21)
(57839, 21)
(57075, 21)
(58986, 21)
(55421, 21)
(54803, 21)
(42867, 21)
(69806, 21)
(54112, 21)
